참조

[Thanks to..](https://wikidocs.net/63565)

#모듈 불러오기

In [1]:
import torch
import torch.nn as nn       #행렬곱
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1109)

if device == 'cuda':
    torch.cuda.manual_seed_all(1109)

#파라미터 설정

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

#데이터 준비

In [4]:
#데이터 다운로드

mnist_train = datasets.MNIST(root='/content/drive/MyDrive/Colab Notebooks',
                             train=True, transform=transforms.ToTensor(),   #텐서로 변환
                             download=True)

mnist_test = datasets.MNIST(root='/content/drive/MyDrive/Colab Notebooks',
                             train=False, transform=transforms.ToTensor(),   #텐서로 변환
                             download=True)

DataLoader로 배치사이즈만큼 데이터 불러오기

In [5]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

데이터 하나씩 확인

In [6]:
dataiter = iter(data_loader)

images, labels = next(dataiter)

In [7]:
#batch_size 100, 이미지의 size는 (1, 28, 28)
images.size()

torch.Size([100, 1, 28, 28])

#모델 설계

1st : conv - relu - maxpool

2nd : conv - relu - maxpool

3rd : fclayer

In [8]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째 층
        # 1개의 입력채널을 (3,3)필터 32개로 convolution
        # relu, maxpooling 시행..
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2),
        )

        # 두번째 층
        # 32개의 입력채널을 (3,3)필터 64개로 convolution
        # relu, maxpooling 시행..
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2),
        )

        # 세번째 층
        # 두번째 층의 output : (batch_size, 64, 7, 7).. -> flatten하면 64*7*7..
        self.layer3 = torch.nn.Linear(64*7*7, 10, bias=True)

        # fclayer Weight Initialization
        torch.nn.init.xavier_normal_(self.layer3.weight)
    

    #Forward computation
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)     #flatten for FC layer
        out = self.layer3(out)
        return out

#모델 정의

In [9]:
model = CNN().to(device)

Cost function : 크로스 엔트로피

Optimizer : Adam

In [10]:
criterion = torch.nn.CrossEntropyLoss().to(device)      #cost에 소프트맥스 포함

#Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#훈련

In [11]:
for epoch in range(training_epochs):
    avg_cost = 0

    #배치 단위로 꺼내온다.. X가 데이터, Y가 레이블
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()       #gradient 초기화
        hypothesis = model(X)       #모델 통과, score획득
        cost = criterion(hypothesis, Y)     #loss 획득
        cost.backward()             #cost를 최적화하도록 역전파
        optimizer.step()

        avg_cost += cost / len(data_loader)

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.217762366
[Epoch:    2] cost = 0.060595952
[Epoch:    3] cost = 0.0439961106
[Epoch:    4] cost = 0.0358495265
[Epoch:    5] cost = 0.0298545044
[Epoch:    6] cost = 0.0248442572
[Epoch:    7] cost = 0.0207025371
[Epoch:    8] cost = 0.018025348
[Epoch:    9] cost = 0.0150358547
[Epoch:   10] cost = 0.0126894442
[Epoch:   11] cost = 0.01177877
[Epoch:   12] cost = 0.00918921083
[Epoch:   13] cost = 0.00859730877
[Epoch:   14] cost = 0.00678968709
[Epoch:   15] cost = 0.00517797843


#테스트

In [12]:
# 학습 끝 -> torch.no_grad()

with torch.no_grad():
    #mnist_test에서 데이터를 shape을 바꿔서 준비까지..
    X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.targets.to(device)

    prediction = model(X_test)      #예측

    #10000개의 데이터에 대한 10개의 클래스일 score가 저장 되어있음
    print(f"테스트세트에 대한 예측점수의 size는 {prediction.size()}")

    #행을 기준으로 max score를 가진 인덱스가 label과 같으면 맞게 예측
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    print(f"테스트세트에 대한 예측의 size는 {correct_prediction.size()}")

    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

테스트세트에 대한 예측점수의 size는 torch.Size([10000, 10])
테스트세트에 대한 예측의 size는 torch.Size([10000])
Accuracy: 0.9876999855041504
